In [ ]:
from model import get_tb_model

In [ ]:
model_config = {
    "start_time": 1850,
    "end_time": 2050,
    "seed": 100,   
}

In [ ]:
studies_dict = {
    "majuro": {
        "pop_size": 27797,
    },
    "study_2": {
        "pop_size": 1.e6,
    }    
}

In [ ]:
model = get_tb_model(model_config, studies_dict)

In [ ]:
params = {
    # Study-specific parameters
    'transmission_rateXmajuro': 10,
    'transmission_rateXstudy_2': 10,

    'lifelong_activation_riskXmajuro': .15,
    'lifelong_activation_riskXstudy_2': .10,
    'prop_early_among_activatorsXmajuro': .90,
    'prop_early_among_activatorsXstudy_2': .90,

    'current_passive_detection_rate': 1.,

    # Universal parameters
    'mean_duration_early_latent': .5,
    'rr_reinfection_latent_late': .2,
    'rr_reinfection_recovered': 1.,
    'self_recovery_rate': .2,
    'tb_death_rate': .2,
    'tx_duration': .5,
    'tx_prop_death': .04
}
model.run(params)

In [ ]:
do_df = model.get_derived_outputs_df()
# do_df[['ltbi_propXstudy_1', 'ltbi_propXstudy_2']].plot()
# do_df[['populationXstudy_1', 'populationXstudy_2']].plot()

In [ ]:
do_df[['ltbi_propXmajuro']].plot()

# Calibration

In [ ]:
from estival import priors as esp
from estival import targets as est
from estival.model import BayesianCompartmentalModel

In [ ]:
def get_priors(studies_dict):

    priors = [
        esp.UniformPrior("current_passive_detection_rate", [.1, 10.]),
    ]

    hyper_mean_lifelong = esp.UniformPrior("hyper_mean_lifelong", [0., 1.])
    hyper_sd_lifelong = esp.UniformPrior("hyper_sd_lifelong", [0., 10.])
    hyper_mean_early = esp.UniformPrior("hyper_mean_early", [0., 1.])
    hyper_sd_early = esp.UniformPrior("hyper_sd_early", [0., 10.])
    for study in studies_dict:
        priors.extend(
            [
                esp.UniformPrior(f"transmission_rateX{study}", [1., 30.]),

                # the two priors below will eventually be TruncNormal distributions linked through hyper-prior distributions (uniforms) 
                esp.TruncNormalPrior(f"lifelong_activation_riskX{study}", hyper_mean_lifelong, hyper_sd_lifelong, [0., 1.]),
                esp.TruncNormalPrior(f"prop_early_among_activatorsX{study}", hyper_mean_early, hyper_sd_early, [0., 1.]),

            ]
        )
    return priors

In [ ]:
priors = get_priors(studies_dict)

In [ ]:
import pandas as pd
targets = [
    est.NormalTarget("ltbi_propXmajuro", data=pd.Series(data=[.38], index=[2018]), stdev=esp.UniformPrior("std_ltbi", [.01, 0.5])),
    est.NormalTarget("tb_prevalence_per100kXmajuro", data=pd.Series(data=[1366], index=[2018]), stdev=esp.UniformPrior("std_tb", [10., 1000.])),
    est.NormalTarget("raw_notificationsXmajuro", data=pd.Series(data=[100], index=[2015]), stdev=esp.UniformPrior("std_not", [1., 100.])),
]

In [ ]:
import nevergrad as ng
from estival.wrappers.nevergrad import optimize_model

def calibrate_with_opti(bcm, n_iter, opt_class=ng.optimizers.NGOpt):
    
    orunner = optimize_model(bcm, opt_class=opt_class)
    rec = orunner.minimize(n_iter)
    mle_params = rec.value[1]

    return mle_params

import matplotlib.pyplot as plt

def visualise_mle_fit(bcm, mle_params):
    print("Running model with MLE parameters...")
    res = bcm.run(mle_params)
    print("... run completed.")
    for target in bcm.targets:
        plt.figure()
        bcm.targets[target].data.plot(style='.')
        res.derived_outputs[target].plot()
        plt.title(target)

In [ ]:
bcm = BayesianCompartmentalModel(model, params, priors, targets)

In [ ]:
# mle_params = calibrate_with_opti(bcm, 200)

In [ ]:
# visualise_mle_fit(bcm, mle_params)

In [ ]:

from estival.wrappers import pymc as epm

import pymc as pm

In [ ]:
with pm.Model() as model:
    
    # This is all you need - a single call to use_model
    variables = epm.use_model(bcm)
    idata = pm.sample(step=[pm.DEMetropolis(variables)], draws=100, tune=0,cores=4,chains=4)